In [1]:
import re
import copy
import time
import math

types=['ore','clay','obsidian','geode']

#print(types)
#print(T)

def parseBP(l):
    #print(l)
    bp={}
    l=l.split(' Each ')
    ix=int(re.findall('[0-9]+',l[0])[0])
    for i in l[1:]:
        t,c=i.split(' robot costs ')
        bp[t]={}
        for c in c[:-1].split(' and '):
            n,p=c.split(' ')
            n=int(n)
            bp[t][p]=int(n)
    return ix,bp



_,tbp1 = parseBP("Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.")
_,tbp2 = parseBP("Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.")



def ld():
    bps={}
    for l in open("19.txt").read().split('\n'):
        ix,bp=parseBP(l)
        #print(l)
        #print(ix)
        #print(bp)
        #assert(0)
        bps[ix]=bp
        #break
    return bps



def MG(bp,maxt):
    #robots
    r={ k:0 for k in types}
    r[types[0]]+=1
    
    #ingredients
    i={ k:0 for k in types}
    
    #robots,ingredients,focus,time
    e=[(r,i,0)]
    
    #compute maximum ingredient cost to prevent overproduction
    mxr={ k:0 for k in types}
    for tp in bp.keys():
        for ing in bp[tp].keys():
            if bp[tp][ing]>mxr[ing]:
                mxr[ing]=bp[tp][ing]
    #print("mxr",mxr)
    
    #record max geodes seen
    maxgeodes=0
    
    #count steps
    cnt = 0
    
    #keep track of time
    lt = time.time()
    
    
    
    #keep exploring
    while len(e):
        #grab situation
        n=e[0]
        e=e[1:]
        
        
        #print status 
        nw=time.time()
        if nw > lt+1:
            lt=nw
            print("%d %d %d           "%(t, len(e), cnt), end="\r")
        
        
        #robots, ingredients, time, focus
        r,i,t=n 

        
         
        #list options
        options = []
        for tp in bp.keys():
            delays=[]
            for ing in bp[tp].keys():
                #can buy now
                if i[ing] >= bp[tp][ing]:
                    delays.append(0)
                #can buy later
                elif r[ing] > 0:
                    short=bp[tp][ing]-i[ing]
                    savetime=short/r[ing]
                    savetime=int(math.ceil(savetime))
                    delays.append(savetime)
                else:
                    delays.append(None)
            
            if not None in delays:
                if r[tp]<mxr[tp] or tp==types[-1]:
                    options.append((tp,max(delays)+1))
        
        
        for tp,delay in options:
            t2=t+delay
            if t2<=maxt:
                r2=copy.deepcopy(r)
                i2=copy.deepcopy(i)
                
                #accumulate production
                for k in i2.keys():
                    i2[k]+=r2[k]*delay

                #purchase robot
                for ing in bp[tp].keys():
                    i2[ing] -= bp[tp][ing]

                #get robot at end of turn
                r2[tp]+=1
                
                #estimate best case geode yield
                theo=i2[types[-1]]
                jj=r2[types[-1]]
                for j in range(maxt-t2):
                    theo+=jj
                    jj+=1
                
                #pickup geodes
                remain=maxt-t2
                maxgeodes=max(i2[types[-1]]+r2[types[-1]]*remain,maxgeodes)
                
                    
                #best case must be better than record    
                if theo>maxgeodes:
                    e.append((r2,i2,t2))
                
                    
                    
        #dive for geodes fast to find something to filter by        
        e=sorted(e,key=lambda x: x[-1],reverse=True)
        cnt += 1 
           
        
    return maxgeodes

print("\ntest1:", MG(tbp1, 24), 9)
print("\ntest2:", MG(tbp2, 24), 12)

   



test1: 9 9

test2: 12 12


In [2]:
 
    

def p1():
    r=0
    bps=ld()
    for ix in bps.keys():
        mg=MG(bps[ix], 24)
        r+=ix*mg
        print("\n%d : %d"%(ix,mg))
        
    return r
    
print("\npart1:",p1(),1266)




1 : 0

2 : 1

3 : 2

4 : 0

5 : 1

6 : 0

7 : 7

8 : 9

9 : 3

10 : 0

11 : 0

12 : 0

13 : 1

14 : 0

15 : 1

16 : 2

17 : 6

18 : 5

19 : 3

20 : 1

21 : 0

22 : 1

23 : 15

24 : 1

25 : 3

26 : 0

27 : 1

28 : 7

29 : 3

30 : 0

part1: 1266 0


In [5]:
def p2():
    r=1
    bps=ld()
    for ix in bps.keys():
        mg=MG(bps[ix], 32)
        r*=mg
        print("\n%d : %d"%(ix,mg))
        if ix==3:
            break
        
    return r
    
print("\npart2:",p2(),0)


27 8 127908           
1 : 10
25 9 87972           
2 : 20
27 4 519650            
3 : 29

part2: 5800 0
